In [334]:
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import os
from sklearn.metrics import roc_auc_score
import numpy as np
from torch.utils.data import TensorDataset
import torch.nn as nn
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import math
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [335]:
class TensorDataset(Dataset):
    def __init__(self, features, labels, device=None):
        self.features = features
        self.labels = labels
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx].to(self.device)
        label = self.labels[idx].to(self.device)
        return feature, label


In [336]:
class AutoEncoder(nn.Module):
    def __init__(self, input_features, output_features, dropout_rate=0.5):
        super(AutoEncoder, self).__init__()

        self.input_layer = nn.Linear(input_features, 256)
        self.batch_norm1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.encoder_layer_1 = nn.Linear(256, 128)
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.bottleneck_layer = nn.Linear(128, 64)

        self.decoder_layer_1 = nn.Linear(64, 128)
        self.batch_norm3 = nn.BatchNorm1d(128)
        self.dropout3 = nn.Dropout(dropout_rate)
        self.decoder_layer_2 = nn.Linear(128, 256)
        self.batch_norm4 = nn.BatchNorm1d(256)
        self.dropout4 = nn.Dropout(dropout_rate)
        self.output_layer = nn.Linear(256, output_features)

    def forward(self, x):
        x = torch.relu(self.batch_norm1(self.input_layer(x)))
        x = self.dropout1(x)
        x = torch.relu(self.batch_norm2(self.encoder_layer_1(x)))
        x = self.dropout2(x)
        x = torch.relu(self.bottleneck_layer(x))  

        x = torch.relu(self.batch_norm3(self.decoder_layer_1(x)))
        x = self.dropout3(x)
        x = torch.relu(self.batch_norm4(self.decoder_layer_2(x)))
        x = self.dropout4(x)

        return self.output_layer(x)

    def encode(self, x):
        x = torch.relu(self.batch_norm1(self.input_layer(x)))
        x = self.dropout1(x)
        x = torch.relu(self.batch_norm2(self.encoder_layer_1(x)))
        x = self.dropout2(x)
        return self.bottleneck_layer(x)

In [337]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_features, dropout_rate=0.5):
        super(BinaryClassifier, self).__init__()

        self.fc1 = nn.Linear(input_features, 128)  
        self.dropout1 = nn.Dropout(dropout_rate) 
        self.fc2 = nn.Linear(128, 64) 
        self.dropout2 = nn.Dropout(dropout_rate) 
        self.fc3 = nn.Linear(64, 32)  
        self.dropout3 = nn.Dropout(dropout_rate) 
        self.fc4 = nn.Linear(32, 1)  
        self.sigmoid = nn.Sigmoid() 

    def forward(self, x):
        x = torch.relu(self.fc1(x)) 
        x = self.dropout1(x)  
        x = torch.relu(self.fc2(x)) 
        x = self.dropout2(x)
        x = torch.relu(self.fc3(x)) 
        x = self.dropout3(x) 
        x = self.fc4(x) 
        x = self.sigmoid(x)  
        return x



In [338]:
def save_results_to_csv(algorithm, train_loss, test_loss, accuracy, auc, file_name,description,disease,valid = False):
    result = {
        'Algorithm': algorithm,
        'Disease': disease,
        'Valid': valid,
        'Train Loss': train_loss,
        'Test Loss': test_loss,
        'Test Accuracy': accuracy,
        'AUC': auc,
        'Timestamp': datetime.now(),
        'description':description
    }
    file_path = 'results\\' + file_name 
    if not os.path.exists(file_path):
        results_df = pd.DataFrame([result])
        results_df.to_csv(file_path, index=False)
    else:
        results_df = pd.DataFrame([result])
        results_df.to_csv(file_path, mode='a', header=False, index=False)

def train_classifier(model, train_loader, test_loader, loss_fn, optimizer, disease, algorithm, description, device=device, epochs=1000, patience=50, min_delta=0):
    best_test_loss = np.inf
    epochs_without_improvement = 0  

    for epoch in range(epochs):
        model.train()
        train_loss = 0

        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(features)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        all_labels = []
        all_outputs = []

        with torch.no_grad():
            for features, labels in test_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = loss_fn(outputs, labels)
                test_loss += loss.item()

                all_labels.extend(labels.cpu().numpy())
                all_outputs.extend(outputs.cpu().numpy())

                predicted = (outputs >= 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_loss /= len(test_loader)
        accuracy = 100 * correct / total
        auc = roc_auc_score(all_labels, all_outputs)

        print(f"Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {train_loss:.4f} "
              f"Test Loss: {test_loss:.4f} "
              f"Test Accuracy: {accuracy:.2f}% "
              f"AUC: {auc:.4f}")

        if test_loss < best_test_loss - min_delta:
            best_test_loss = test_loss
            epochs_without_improvement = 0  
            best_model_state = model.state_dict() 
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            save_results_to_csv(algorithm,
                                train_loss=train_loss,
                                test_loss=test_loss,
                                accuracy=accuracy,
                                auc=auc,
                                file_name='result.csv',
                                disease=disease, valid=True,
                                description=description)
            break

        if epoch == epochs - 1 or epochs_without_improvement >= patience:
            save_results_to_csv(algorithm,
                                train_loss=train_loss,
                                test_loss=test_loss,
                                accuracy=accuracy,
                                auc=auc,
                                file_name='result.csv',
                                disease=disease, valid=True,
                                description=description)

    if 'best_model_state' in locals():
        model.load_state_dict(best_model_state)

    return model


In [339]:
def apply_log_transform(features):
    return np.log(features + 1e-6) 

def apply_variance_thresholding(feature,variance_threshold):
    feature_variances = np.var(feature, axis=0) 
    feature = feature[:, feature_variances < variance_threshold]
    return feature

def select_top_k_feature_from_random_forest(n_estimators,feature,ratio,df):
        rf = RandomForestClassifier(n_estimators, random_state=42)
        rf.fit(feature, df['disease'].apply(lambda x: 0 if x == 'healthy' else 1))
        feature_importances = rf.feature_importances_
        total_features = feature.shape[1]
        k = math.floor((total_features*ratio))  
        top_k_features = np.argsort(feature_importances)[-k:]
        feature = feature[:, top_k_features]
        return feature
    
def select_feature_from_autoencoder(model, feature, epochs=500, device=device, patience=10):

    model.train()
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    feature_tensor = torch.Tensor(feature).to(device=device)

    best_loss = float('inf')
    patience_counter = 0
    for epoch in range(epochs):
        acc_loss = 0
        for batch in DataLoader(feature_tensor, batch_size=64, shuffle=True):
            optimizer.zero_grad()
            batch = batch.to(device=device)
            output = model(batch)
            loss = loss_fn(output, batch)
            acc_loss += loss.item()
            loss.backward()
            optimizer.step()

        avg_loss = acc_loss / len(feature_tensor) 
        
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.5f}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0  
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break

    model.eval()
    encoded_features = model.encode(feature_tensor)
    return encoded_features.detach().cpu().numpy()


In [340]:
#-----------Define log----------------------
algorithm = 'log_transform+normalization+random_forest+early_stopping'
description = '500 trees, k = 3/4'

#-----------Define disease file map---------------
disease_file_map = {
    'Cirrhosis': 'cirrhosis_10fold.csv',
    'CRC': 'CRC_10fold.csv',
    'CRC_AUS': 'CRC_AUS_LOSO.csv',
    'CRC_CHI': 'CRC_CHI_LOSO.csv',
    'CRC_FRA': 'CRC_FRA_LOSO.csv',
    'CRC_GER': 'CRC_GER_LOSO.csv',
    'CRC_IND': 'CRC_IND_additional.csv',
    'CRC_USA': 'CRC_USA_LOSO.csv',
    'IBD': 'IBD_10fold.csv',
    'IBD_DK': 'IBD_DK_LOSO.csv',
    'IBD_UK': 'IBD_UK_LOSO.csv',
    'Obt': 'Obt_10fold.csv',
    'T2D': 'T2D_10fold.csv'
}

for disease, file_name in disease_file_map.items():
    
    for iteration in range(5):
        
        print('-------------' + 'disease: ' + disease + ' | ' + 'iteration: ' + str(iteration))

    #---------- read data from csv-----------------
        raw_df = pd.read_csv('data\\' + disease_file_map[disease])
        raw_df['disease'] = raw_df['disease'].apply(lambda x: 0 if x == 'healthy' else 1)
        feature = raw_df.iloc[:, 4:].values 

        # ---------- feature selection algorithm -----------------
        # 1. Log Transformation (Optional)
        feature = apply_log_transform(features=feature)

        # # 2. Variance Thresholding
        # feature = apply_variance_thresholding(feature=feature,variance_threshold=0.2)

        # 3. apply auto encoder 
        # input_features = feature.shape[1]
        # autoencoder = AutoEncoder(input_features=input_features, output_features=input_features).to(device=device)
        # encoded_features = select_feature_from_autoencoder(model=autoencoder, feature=feature)
        # encoded_features = select_top_k_feature_from_random_forest(n_estimators=1000,feature=feature,ratio=3/4,df=raw_df)

        # 4. Normalization (Min-Max scaling between 0 and 1)
        feature_min = np.min(feature, axis=0)
        feature_max = np.max(feature, axis=0)
        feature = (feature - feature_min) / (feature_max - feature_min + 1e-6)

        feature = select_top_k_feature_from_random_forest(n_estimators=500,feature=feature,ratio=(3/4),df=raw_df)

        #---------- convert label and feature into tensor-----------------
        label = torch.tensor(raw_df.loc[:, 'disease'].values).unsqueeze(1).float().to(device=device) 
        feature = torch.Tensor(feature).to(device=device)


        #----------split tensor into training set and test set-----------------
        feature_train, feature_test, label_train, label_test = train_test_split(feature, label, test_size=0.2)

        #----------define data loader------------------
        train_loader =  DataLoader(TensorDataset(feature_train,label_train), batch_size=64, shuffle=True)
        test_loader = DataLoader(TensorDataset(feature_test,label_test), batch_size=64, shuffle=True)

        #----------define classifier model-----------------
        classifier = BinaryClassifier(input_features=feature_train.shape[1]).to(device=device)


        #---------define loss function and optimizer----------
        loss_fn = nn.BCELoss()
        optimizer = torch.optim.Adam(
            classifier.parameters(),
            lr=0.0001,
            betas=(0.9, 0.999),
            eps=1e-8,
            weight_decay=1e-5
        )

        #---------train the classifier model-------------
        train_classifier(
            model=classifier,
            train_loader=train_loader,
            test_loader=test_loader,
            epochs=1000,
            loss_fn=loss_fn,
            optimizer=optimizer,
            disease=disease,
            algorithm=algorithm,
            description = description,
    )

-------------disease: Cirrhosis | iteration: 0
Epoch [1/1000] Train Loss: 0.6934 Test Loss: 0.7037 Test Accuracy: 41.67% AUC: 0.3911
Epoch [2/1000] Train Loss: 0.6963 Test Loss: 0.7033 Test Accuracy: 41.67% AUC: 0.4911
Epoch [3/1000] Train Loss: 0.6924 Test Loss: 0.7028 Test Accuracy: 41.67% AUC: 0.5714
Epoch [4/1000] Train Loss: 0.6941 Test Loss: 0.7023 Test Accuracy: 41.67% AUC: 0.6375
Epoch [5/1000] Train Loss: 0.6932 Test Loss: 0.7018 Test Accuracy: 41.67% AUC: 0.6929
Epoch [6/1000] Train Loss: 0.6928 Test Loss: 0.7013 Test Accuracy: 41.67% AUC: 0.7500
Epoch [7/1000] Train Loss: 0.6930 Test Loss: 0.7009 Test Accuracy: 41.67% AUC: 0.7929
Epoch [8/1000] Train Loss: 0.6998 Test Loss: 0.7004 Test Accuracy: 41.67% AUC: 0.8482
Epoch [9/1000] Train Loss: 0.6938 Test Loss: 0.7000 Test Accuracy: 41.67% AUC: 0.8839
Epoch [10/1000] Train Loss: 0.6940 Test Loss: 0.6994 Test Accuracy: 41.67% AUC: 0.9054
Epoch [11/1000] Train Loss: 0.6950 Test Loss: 0.6989 Test Accuracy: 41.67% AUC: 0.9161
Epoch

In [346]:

# Load the data
data = pd.read_csv('results\\result.csv')

# Define the algorithms (including the new one)
algorithms = [
    'raw+early_stopping', 
    'log_transform+normalization+early_stopping', 
    'log_transform+variance_threshold+normalization+early_stopping+random_forestV2',
    'log_transform+variance_threshold+auto_encoder+normalization+early_stopping'
]

# Colors for each algorithm; use dark green for the AE bar, blue for AE error line
colors = ['black', 'red', '#006400', 'darkblue']
error_colors = ['red', 'black', 'blue', 'orange']  # Error color for LVN-RF2

# Get unique diseases
diseases = data['Disease'].unique()

# Split diseases into three groups
disease_groups = [
    diseases[:4],   # First 4 diseases
    diseases[4:8],  # Next 4 diseases
    diseases[8:]    # Remaining diseases (5)
]

def create_single_group_plot(disease_group_data, diseases_in_group, group_idx):
    num_diseases = len(diseases_in_group)
    
    # Create the plot with sufficient space for the groups
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Set the background to white
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    
    positions = np.arange(num_diseases)  # The x locations for the diseases
    width = 0.2  # The width of the bars (adjusted for four bars)

    # Lists to store means and std deviations for each algorithm
    algorithm_means = [[] for _ in algorithms]
    algorithm_stds = [[] for _ in algorithms]

    for disease in diseases_in_group:
        disease_data = disease_group_data[disease_group_data['Disease'] == disease]
        
        # Calculate statistics for each algorithm
        for algo_idx, algo in enumerate(algorithms):
            algo_data = disease_data[disease_data['Algorithm'] == algo]
            auc_values = algo_data['AUC'].astype(float)
            mean_auc = auc_values.mean()
            std_auc = auc_values.std()
            
            algorithm_means[algo_idx].append(mean_auc)
            algorithm_stds[algo_idx].append(std_auc)
    
    # Create the bars for each algorithm
    bars = []
    for i, (means, stds, color, err_color) in enumerate(zip(algorithm_means, algorithm_stds, colors, error_colors)):
        bars.append(ax.bar(
            positions + (i - 1.5) * width, means, width, color=color, yerr=stds,
            error_kw=dict(elinewidth=2, ecolor=err_color)
        ))

    # Set the axis labels and title
    ax.set_xlabel('Diseases', color='black')
    ax.set_ylabel('AUC', color='black')
    ax.set_title('AUC Performance Comparison Across Diseases and Algorithms', color='black')
    ax.set_xticks(positions)
    ax.set_xticklabels(diseases_in_group, rotation=45, color='black')
    ax.set_ylim(0, 1)
    
    # Ensure axis lines and ticks are visible
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    
    # Make sure all text is black
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_color('black')
    
    # Add the text labels inside the bars (centered horizontally and vertically) with rotation
    algo_labels = ['R', 'LN', 'LVN-RF', 'LVN-AE']
    for i, (bar_group, label) in enumerate(zip(bars, algo_labels)):
        for rect in bar_group:
            height = rect.get_height()
            ax.text(
                rect.get_x() + rect.get_width() / 2, height / 2, label,
                ha='center', va='center', color='white', fontweight='bold', rotation=90  # Rotate label by 90 degrees
            )
            # Add the maximum value at the top of the bar
            ax.text(
                rect.get_x() + rect.get_width() / 2, height + 0.02, f'{height:.2f}',
                ha='center', va='bottom', color='black', fontweight='bold', fontsize=8  # Adjust fontsize as needed
            )
    
    # Save the figure
    os.makedirs('graphs', exist_ok=True)
    fig.savefig(f'{"graphs"}/auc_comparison_group_{group_idx}.png', bbox_inches='tight', dpi=300)
    
    # Show the plot
    plt.close(fig)

# Create plots for each disease group
for group_idx, disease_group in enumerate(disease_groups):
    group_data = data[data['Disease'].isin(disease_group)]
    create_single_group_plot(group_data, disease_group, group_idx)
